# **Laboratorio 9: Airflow 🛫**

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2025</strong></center>

### Cuerpo Docente:

- Profesores: Diego Cortez, Gabriel Iturra
- Auxiliares: Melanie Peña, Valentina Rojas
- Ayudantes: Nicolás Cabello, Cristopher Urbina

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Josefa Anselmo.
- Nombre de alumno 2: Tamara Carrasco.

### **Link de repositorio de GitHub:** [TamoJosha](https://github.com/Tamaracarrasco/TamoJosha-Lab-de-Prog-MDS-Prim2025/)

## Temas a tratar

- Construcción de pipelines productivos usando `Airflow`.

## Reglas:

- **Grupos de 2 personas**
- Fecha de entrega: Entregas Martes a las 23:59.
- Instrucciones del lab el viernes a las 16:15 en formato online. Asistencia no es obligatoria, pero se recomienda **fuertemente** asistir.
- <u>Prohibidas las copias</u>. Cualquier intento de copia será debidamente penalizado con el reglamento de la escuela.
- Tienen que subir el laboratorio a u-cursos y a su repositorio de github. Labs que no estén en u-cursos no serán revisados. Recuerden que el repositorio también tiene nota.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Pueden usar cualquier material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Reconocer los componentes pricipales de `Airflow` y su funcionamiento.
- Poner en práctica la construcción de pipelines de `Airflow`.
- Automatizar procesos típicos de un proyecto de ciencia de datos mediante `Airflow` y `Docker`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# **Introducción**

<p align="center">
  <img src="https://media.tenor.com/OBQ6niqbxswAAAAM/legallyblonde.gif" width="300">
</p>

Vale, una estudiante del Magíster en Ciencia de Datos, se encuentra en la etapa final de sus estudios. Por un lado, está muy contenta por haber llegado tan lejos, pero por otro, no puede evitar sentirse inquieta. Desde que ingresó a la universidad, una pregunta la ha perseguido: ¿qué tan probable es que pueda ser seleccionada en los lugares donde envíe postulaciones para puestos de trabajo?

Esta duda la mantiene en constante reflexión, especialmente porque sabe que el mercado laboral en Ciencia de Datos es competitivo y exige habilidades no solo técnicas, sino también estratégicas para destacar. Sin embargo, Vale actualmente está completamente enfocada en terminar su tesis de magíster y ha tenido que postergar cualquier preparación específica para enfrentar el desafío de las postulaciones laborales.

Al ver el avance y las habilidades que usted ha demostrado en el curso, Vale decidió proponerle un desafío que le permitirá disminuir la incertidumbre sobre su futuro laboral. Inspirado en sus conocimientos, recolectó un conjunto de datos que contiene información sobre diversos factores que influyen en las decisiones de contratación de empresas al seleccionar entre sus postulantes. Este set de datos incluye los siguientes atributos:

- Age: Edad del candidato
- Gender: Genero del candidato. Male (0), Female (1).
- EducationLevel: Mayor nivel educacional alcanzado por el candidato. Licenciatura Tipo 1 (1), Licenciatura Tipo 2 (2), Maestría (3), PhD. (4).
- ExperienceYears: Años de experiencia profesional.
- PreviousCompanies: Numero de compañías donde el candidato ha trabajado anteriormente.
- DistanceFromCompany: Distancia en kilometros entre la residencia del candidato y la compañía donde postula.
- InterviewScore: Puntaje obtenido en la entrevista por el candidato entre 0 a 100.
- SkillScore: Puntaje obtenido en evaluación de habilidades técnicas por el candidato, entre 0 a 100.
- PersonalityScore: Puntaje obtenido en pruebas de personalidad del candidato, entre 0 a 100.
- RecruitmentStrategy: Estrategia del equipo de reclutamiento. Agresiva (1), Moderada (2), Conservadora (3).

Variable a predecir:
- HiringDecision: Resultado de la postulación. No contratado (0), Contratado (1).

Su objetivo será ayudar a Vale a desarrollar un modelo que le permita predecir, basado en estos factores, si un postulante será contratado o no. Esta herramienta no solo le dará a Vale mayor claridad sobre el impacto de ciertos atributos en la decisión final de contratación, sino que también le permitirá aplicar sus conocimientos de Ciencia de Datos para resolver una pregunta que a muchos estudiantes como a ella les inquieta.

Como estudiante del curso Laboratorio de Programación Científica para Ciencia de Datos, deberá demostrar sus capacidades para preprocesar, analizar y modelar datos, brindándole a Vale una solución robusta y bien fundamentada para su problemática.

`Nota:` El siguiente [enlace](https://www.kaggle.com/datasets/rabieelkharoua/predicting-hiring-decisions-in-recruitment-data/data) contiene el set de datos original.

# **1. Pipeline de Predicción Lineal** (30 Puntos)

<p align="center">
  <img src="https://media.licdn.com/dms/image/v2/D4E22AQHZplrdPyKnvA/feedshare-shrink_2048_1536/feedshare-shrink_2048_1536/0/1713736729086?e=2147483647&v=beta&t=Tad2ulaWkhhDrPRN0PCdXrfuza60PjoJqgLborDyLao" width="500">
</p>

En esta sección buscaremos desplegar un producto utilizando un modelo de clasificación `Random Forest` para determinar **si una persona será contratada o no en un proceso de selección**. Para ello, comenzaremos preparando un pipeline lineal mediante `Airflow`.

## **1.1 Preparando el Pipeline** (15 puntos)

**Primero, asegúrese de tener creada las carpetas `dags`, `plugins` y `logs`**.

Comenzamos preparando un archivo llamado `hiring_functions.py`, el cual guardará en la carpeta `dags` y debe contener lo siguiente:

1. (3 puntos) Una función llamada `create_folders()` que cree una carpeta, la cual utilice la fecha de ejecución como nombre. Adicionalmente, dentro de esta carpeta debe crear las siguientes subcarpetas:
  - raw
  - splits
  - models

  `Hint`: Puede hacer uso de kwargs para obtener la fecha de ejecución mediante el DAG. El siguiente [Enlace](https://airflow.apache.org/docs/apache-airflow/stable/templates-ref.html) le puede ser útil.

2. (3 puntos) Una función llamada `split_data()` que lea el archivo `data_1.csv` de la carepta `raw` y a partir de este, aplique un *hold out*, generando un dataset de entrenamiento y uno de prueba. Luego debe guardar estos nuevos conjuntos de datos en la carpeta `splits`. `Nota:` Utilice un 20% para el conjunto de prueba, mantenga la proporción original en la variable objetivo y fije una semilla.

3. (8 puntos) Cree una función llamada `preprocess_and_train()` que:
  - Lea los set de entrenamiento y prueba de la carpeta `splits`.
  - Cree y aplique un `Pipeline` con una etapa de preprocesamiento. Utilice `ColumnTransformers` para aplicar las transformaciones que estime convenientes. Puede apoyarse del archivo `data_1_report.html` para justificar cualquier paso del preprocesamiento.
  
  - Añada una etapa de entrenamiento utilizando el modelo `RandomForest`.
  
  Esta función **debe crear un archivo `joblib` (análogo a `pickle`) con el pipeline entrenado** en la carepta `models`, además debe **imprimir** el accuracy en el conjunto de prueba y el f1-score de la clase positiva (contratado).
  
3. (1 punto) Incorpore la función `gradio_interface` en su script, modificando la ruta de acceso a su modelo, de forma que pueda leerlo desde la carepta `models`. Puede realizar las modificaciones que estime necesarias.

`NOTA:` Se permite la creación de funciones auxiliares si lo estiman conveniente.

In [ ]:
#Inserte su código aqui

# Este es el código de mi archivo hiring_functions.py

## Desarrollo parte 1.1 Preparando el Pipeline


## creación de las carpetas
# Primero me ubico donde quiero crearlas
cd Laboratorios\Laboratorio_9

# y las creo --> Por temas de compatibilidad con el tipo de archivo que se puede subir a GIT, tuvimos qeu crear un gitignor que "elimina" de repo
# remoto las carpetas de logs y plugins, pero en local las necesitamos, por eso se podría  crear con el comando solo esas dos y no dags por esa si permite subirlo
mkdir -Force dags, plugins, logs

# Luego creo el archivo dentro de dags
New-Item -Path ".\dags\hiring_functions.py" -ItemType "File" -Force


# hiring_functions.py

# Librerías
from pathlib import Path
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
import joblib


# Primera definición: create_folders(**context)

def create_folders(**context):
    """
    Crea la carpeta de ejecución YYYY-MM-DD con subcarpetas raw/splits/models
    """
    ds = context["ds"]                             
    dags_dir = Path(__file__).resolve().parent    
    run_dir  = dags_dir / ds

    for sub in ["raw", "splits", "models"]:
        (run_dir / sub).mkdir(parents=True, exist_ok=True)
        

# Segunda definición: split_data(**context)--> hold-out 80/20 estratificado

def split_data(**context):
    """
    Lee <ds>/raw/data_1.csv, aplica hold-out 80/20 estratificado (seed=42)
    y guarda train/test en <ds>/splits.
    """
    ds = context["ds"]
    base = Path(__file__).resolve().parent / ds
    raw_file = base / "raw" / "data_1.csv"

    df = pd.read_csv(raw_file)

    y = df["HiringDecision"]
    X = df.drop(columns=["HiringDecision"])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42, stratify=y
    )

    splits_dir = base / "splits"
    splits_dir.mkdir(parents=True, exist_ok=True)
    X_train.assign(HiringDecision=y_train).to_csv(splits_dir / "train.csv", index=False)
    X_test.assign(HiringDecision=y_test).to_csv(splits_dir / "test.csv", index=False)

    print(f"[split_data] Guardados: {splits_dir / 'train.csv'} y {splits_dir / 'test.csv'}")


# Tercera definición: preprocess_and_train(**context) --> preprocesa, entrena y evalúa

def preprocess_and_train(**context):
    """
    - Lee <ds>/splits/train.csv y test.csv
    - Preprocesa: imputación median (num) / most_frequent + OneHot (cat)
    - Entrena RandomForest
    - Imprime accuracy y f1 (clase positiva=1)
    - Guarda pipeline en <ds>/models/pipeline.joblib
    """
    ds = context["ds"]
    base = Path(__file__).resolve().parent / ds
    splits_dir = base / "splits"
    models_dir = base / "models"

    train_df = pd.read_csv(splits_dir / "train.csv")
    test_df = pd.read_csv(splits_dir / "test.csv")

    y_train = train_df["HiringDecision"]
    X_train = train_df.drop(columns=["HiringDecision"])
    y_test = test_df["HiringDecision"]
    X_test = test_df.drop(columns=["HiringDecision"])

    num_cols = [
        "Age", "ExperienceYears", "PreviousCompanies", "DistanceFromCompany",
        "InterviewScore", "SkillScore", "PersonalityScore"
    ]
    cat_cols = ["Gender", "EducationLevel", "RecruitmentStrategy"]

    num_cols = [c for c in num_cols if c in X_train.columns]
    cat_cols = [c for c in cat_cols if c in X_train.columns]

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])

    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ],
        remainder="drop",
    )

    clf = RandomForestClassifier(
        n_estimators=300,
        random_state=42,
        n_jobs=-1,
    )

    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", clf),
    ])

    # entrenamiento
    pipe.fit(X_train, y_train)

    # evaluación
    y_pred = pipe.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    f1_pos = f1_score(y_test, y_pred, pos_label=1)

    print(f"[preprocess_and_train] Test Accuracy: {acc:.4f}")
    print(f"[preprocess_and_train] Test F1-score (clase 1 - contratado): {f1_pos:.4f}")

    # guardar modelo
    models_dir.mkdir(parents=True, exist_ok=True)
    model_path = models_dir / "pipeline.joblib"
    joblib.dump(pipe, model_path)
    print(f"[preprocess_and_train] Modelo guardado en: {model_path}")


# Cuarta definicion: gradio_interface(**kwargs)--> UI mínima para predecir desde JSON

def predict(file, model_path):
    """
    Carga el modelo y predice a partir de un archivo JSON con las features.
    Soporta gr.File (con .name) o una ruta de archivo (str/Path).
    """
    pipeline = joblib.load(model_path)
    path = file.name if hasattr(file, "name") else file
    input_data = pd.read_json(path)
    preds = pipeline.predict(input_data)
    labels = ["No contratado" if p == 0 else "Contratado" for p in preds]
    return {"Predicción": labels[0]}


def gradio_interface(**kwargs):
    """
    Interfaz Gradio que carga el modelo desde <ds>/models/pipeline.joblib.
    """
    import gradio as gr

    run_folder_name = kwargs["ds"]
    dags_dir = Path(__file__).resolve().parent
    model_path = dags_dir / run_folder_name / "models" / "pipeline.joblib"

    interface = gr.Interface(
        fn=lambda file: predict(file, model_path),
        inputs=gr.File(label="Sube un archivo JSON"),
        outputs="json",
        title="Hiring Decision Prediction",
        description=(
            "Sube un archivo JSON con las características de entrada "
            "para predecir si Vale será contratada o no."
        ),
        allow_flagging="never",
    )

    interface.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=True,    
        inbrowser=False,
    )

## **1.2 Creando Nuestro DAG** (15 puntos)

<p align="center">
  <img src="https://media.tenor.com/a_yibuZQgngAAAAM/elle-woods.gif" width="400">
</p>

Con las funciones del pipeline ya creadas, ahora vamos a proceder a crear un Directed Acyclic Graph (DAG). Para ello, se le pide lo siguiente:

- (10 puntos) Cree un segundo archivo llamado `dag_lineal.py` y guardelo en la carpeta dags. Este script debe seguir la siguiente estructura (Ver imagen de referencia):

    0. Inicialice un DAG con fecha de inicio el 1 de octubre de 2024, ejecución manual y **sin backfill**. Asigne un `dag_id` que pueda reconocer facilmente, como `hiring_lineal`, etc.
    1. Debe comenzar con un marcador de posición que indique el inicio del pipeline.
    2. Cree una carpeta correspondiente a la ejecución del pipeline y cree las subcarpetas `raw`, `splits` y `models` mediante la función `create_folders()`.
    3. Debe descargar el archivo `data_1.csv` del siguiente [enlace](https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv). Debe guardar el archivo en la carpeta raw de la ejecución correspondiente.`Hint:` Le puede ser útil el comando `curl -o <path de guardado> <enlace con los datos>`.
    4. Debe aplicar un hold out mediante la función `split_data()` de su archivo creado en la subsección anterior.
    5. Debe aplicar el preprocesamiento y el entrenamiento del modelo mediante la función `preprocess_and_train()`.
    6. Finalmente, debe montar una interfaz en gradio donde pueda cargar un archivo ``json``.


- (3 puntos) Cree un `DockerFile` para montar un contenedor que contenga Airflow. Adicionalmente, cree una carpeta llamada dags donde guardará el script.py creado anteriormente.

    `Nota:` Para la imagen, se recomienda utilizar python 3.10-slim. Adicionalmente, puede instalar `curl` mediante la siguiente linea de código: `RUN apt-get update && apt-get install -y curl`.

- Construya el contenedor en Docker y acceda a la aplicación web de Airflow mediante el siguiente [enlace](http://localhost:8080/). Inicie sesión, acceda al DAG creado y ejecute de forma manual su pipeline.

- (2 puntos) Acceda a la URL pública de Gradio e ingrese el archivo `vale_data.json` a su modelo. ¿Que predicción entregó el modelo para Vale? Adjunte imágenes de su resultado. `Hint:` Puede acceder a los `logs` para obtener los prints y la URL pública.

`Hint:` Recuerde que puede entregar `kwargs` a sus funciones, como por ejemplo la fecha de ejecución `ds`.

**Para esta sección, debe adjuntar todos los scripts creados junto a su notebook en la entrega, ya que serán ejecutados para validar el funcionamiento. Para justificar sus respuestas, adicionaslmente puede utilizar imágenes de apoyo, como screenshots.**

DAG de referencia:
<p align="center">
  <img src="https://drive.google.com/uc?id=1iwDgECZfFeWq1dl433tMa6_3CNF9cn1L" width="1200">
</p>




In [ ]:
#Inserte código aqui

# creo archivos con comando de powershel para vscode
# Tengo qeu seguir en la carpeta del lab9
cd Laboratorios\Laboratorio_9

creo el archivo
New-Item -Path ".\dags\dag_lineal.py" -ItemType "File" -Force

## dag_lineal.py

# Librerías
from datetime import datetime
from airflow import DAG
from airflow.operators.empty import EmptyOperator
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator
from hiring_functions import (create_folders, split_data, preprocess_and_train, gradio_interface)

# DAG con ejecución manual, sin backfill, start 2024-10-01.

with DAG(
    dag_id="hiring_lineal",
    start_date=datetime(2024, 10, 1),
    schedule=None,         
    catchup=False,        # esto es lo del backfill
    tags=["lab9"],
    is_paused_upon_creation=False,
) as dag:

# Marcador de posición que indica el inicio del pipeline.

    start = EmptyOperator(task_id="start")

# Creación carpetas  para el pipeline y subcarpetas raw, splits y models con la función create_folders()

    mk_run_folders = PythonOperator(
        task_id="create_folders",
        python_callable=create_folders,
        op_kwargs={"ds": "{{ ds }}"},
    )

# Descarga de data_1.csv para guardar en carpeta raw de la ejecución

    download_raw = BashOperator(
    task_id="download_raw",
    bash_command=(
        "mkdir -p {{ dag.folder }}/{{ ds }}/raw && "
        "curl -sSL -o {{ dag.folder }}/{{ ds }}/raw/data_1.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv"
    ),
)

# Aplico hold out con la función split_data()

    do_split = PythonOperator(
        task_id="split_data",
        python_callable=split_data,
        op_kwargs={"ds": "{{ ds }}"},
    )

# Aplico preprocesamiento y entrenamiento del modelo con la función preprocess_and_train()

    train_model = PythonOperator(
        task_id="preprocess_and_train",
        python_callable=preprocess_and_train,
        op_kwargs={"ds": "{{ ds }}"},
    )

# Interfaz de gradio para cargar archivo json

    launch_gradio = PythonOperator(
        task_id="gradio_interface",
        python_callable=gradio_interface,
        op_kwargs={"ds": "{{ ds }}"},
    )

    start >> mk_run_folders >> download_raw >> do_split >> train_model >> launch_gradio

In [ ]:
# Eset es mi código del archivo Dockerfile.dockerfile (no va a correr aca obviamente al no ser .py)

# Me redirecciono a mi carpte
cd Laboratorios\Laboratorio_9

# Creación del dockerfile
New-Item -Path . -Name "Dockerfile" -ItemType "File"



# Dockerfile.dockerfile
# version de python recomendada en el enunciado
FROM python:3.10-slim

ENV DEBIAN_FRONTEND=noninteractive \
    PYTHONUNBUFFERED=1 \
    AIRFLOW_HOME=/opt/airflow

RUN apt-get update && apt-get install -y --no-install-recommends \
    curl build-essential git \
 && rm -rf /var/lib/apt/lists/*

RUN pip install --no-cache-dir "apache-airflow==2.10.2" \
  --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.10.2/constraints-3.10.txt"

RUN pip install --no-cache-dir \
    pandas scikit-learn joblib numpy \
    gradio huggingface_hub

RUN mkdir -p ${AIRFLOW_HOME}/dags ${AIRFLOW_HOME}/logs ${AIRFLOW_HOME}/plugins
WORKDIR ${AIRFLOW_HOME}
COPY dags/ ./dags/

EXPOSE 8080 7860
CMD ["bash", "-lc", "airflow standalone"]


##-------------------------------------------------------------------------------------------------------------------------------------------------------------
## con este comando construyo la imagen desde Power Shell

## CLAVE ABRIR/ENCENDER DOCKER DESKTOP O SIMILAR

# Primero hay que estar parado en la carpeta, si no se esta ahí use este comentario sin el #
# cd Laboratorios\Laboratorio_9

# Tuve que especificar el nmbre de mi archivo pq por defecto los guarda con extension y docker no reconcoa eso me daba error

# Build
docker build -f Dockerfile.dockerfile -t airflow-hiring:latest .

# Limpia contenedor anterior
docker rm -f hiring-airflow 2>$null

# Limpia PIDs
Remove-Item -Force .\airflow_home\airflow-webserver.pid -ErrorAction SilentlyContinue
Remove-Item -Force .\airflow_home\airflow-scheduler.pid -ErrorAction SilentlyContinue

# Run: SOLO dag_lineal + sus funciones utilitarias
docker run --name hiring-airflow `
  -p 8080:8080 -p 7860:7860 `
  -e AIRFLOW__CORE__LOAD_EXAMPLES=False `
  -e AIRFLOW__CORE__EXECUTOR=SequentialExecutor `
  -e AIRFLOW__WEBSERVER__BASE_URL=http://localhost:8080 `
  -v "${PWD}\airflow_home:/opt/airflow" `
  -v "${PWD}\dags\dag_lineal.py:/opt/airflow/dags/dag_lineal.py" `
  -v "${PWD}\dags\hiring_functions.py:/opt/airflow/dags/hiring_functions.py" `
  -v "${PWD}\plugins:/opt/airflow/plugins" `
  airflow-hiring:latest airflow standalone

### luego de esto deberiamos poder acceder a este link:
http://localhost:8080/ # Airflow imprime directaemte als credencialex admin/contraseña

# Después tengo que esperar a que la parte de gradio quede como runn, abrir el log y ahi se ve la web publica que proporciona gradio y te permite ingresar la base de datos

El DAG `hiring_lineal.py` se ejecutó correctamente, permitiendo exponer el modelo entrenado mediante una interfaz pública de **Gradio**, desde la cual se realizaron predicciones individuales y se verificó el desempeño del modelo.

**Flujo del DAG**

![Flujo del DAG del hiring_lineal.py](DAG.png)

**Log de `gradio_interface` – URL pública del modelo**

![Log de gradio_interface](log_gradio_interface.png)

**Resultado de predicción para Vale**

![Predicción modelo Vale](hiring_decision_prediction.png)

**Métricas de desempeño (Accuracy y F1-Score)**

![Accuracy y F1 score](log_y_valores_accu_f1.png)

El modelo indica que con el archivo `vale_data.json`, Vale no sería contratada (0).  
El accuracy del modelo en el conjunto de prueba fue de 0.9267, y el F1-score de la clase positiva (*contratada*) alcanzó 0.8706, mostrando un buen equilibrio entre precisión y recall.  

# **2. Paralelizando el Pipeline** (30 puntos)

<p align="center">
  <img src="https://i.gifer.com/8LNL.gif" width="400">
</p>



Al ver los resultados obtenidos, Vale queda muy contenta con el clasificador. Sin embargo, le aparecen algunas dudas respecto al funcionamiento del pipeline. Primero le comenta que es posible que en un futuro tenga nuevos datos que podrían ser útiles para realizar nuevos entrenamientos, por lo que sería ideal si este pipeline se fuera ejecutando de forma periódica y **NO** de forma manual. Además, Vale le menciona que le gustaría explorar el desempeño de otros modelos además de `Random Forest`, de forma que el pipeline seleccione de forma automática el modelo con mejor desempeño para luego hacer la predicción de Vale.

## **2.1 Preparando un Nuevo Pipeline** (15 puntos)

<p align="center">
  <img src="https://media.tenor.com/gnA7-5TewXMAAAAM/elle-woods.gif" width="400">
</p>

De acuerdo a lo que le comentó Vale, usted decide crear un nuevo script con las funciones que utilizará su pipeline. Por ende, dentro de la carpeta `dags`, usted creará el archivo `hiring_dynamic_functions.py` el cual debe contener:

1. (2 puntos) Una función llamada `create_folders()` que cree una carpeta, la cual utilice la fecha de ejecución como nombre. Adicionalmente, dentro de esta carpeta debe crear las siguientes subcarpetas:
  - raw
  - preprocessed
  - splits
  - models
2. (2 puntos) Una función llamada `load_ands_merge()` que lea desde la carpeta `raw` los archivos `data_1.csv`y `data_2.csv` en caso de estar disponible. Luego concatene estos y genere un nuevo archivo resultante, guardándolo en la carpeta `preprocessed`.

3. (2 puntos) Una función llamada `split_data()` que lea la data guardada en la carpeta `preprocessed` y realice un hold out sobre esta data. Esta función debe crear un conjunto de entrenamiento y uno de prueba. Mantenga una semilla y 20% para el conjunto de prueba. Guarde los conjuntos resultantes en la carpeta `splits`.

4. (6 puntos) Una función llamada `train_model()` que reciba un modelo de clasificación.
    - La función debe comenzar leyendo el conjunto de entrenamiento desde la carpeta `spits`.
    - Esta debe crear y aplicar un `Pipeline` con una etapa de preprocesamiento. Utilice `ColumnTransformers` para aplicar las transformaciones que estime convenientes.
    - Añada una etapa de entrenamiento utilizando un modelo que ingrese a la función.
  
  Esta función **debe crear un archivo joblib con el pipeline entrenado**. Guarde el modelo con un nombre que le permita una facil identificación dentro de la carpeta `models`.

5. (3 puntos) Una función llamada `evaluate_models()` que reciba sus modelos entrenados desde la carpeta `models`, evalúe su desempeño mediante `accuracy` en el conjunto de prueba y seleccione el mejor modelo obtenido. Luego guarde el mejor modelo como archivo `.joblib`. Su función debe imprimir el nombre del modelo seleccionado y el accuracy obtenido.

In [ ]:
#Inserte código aqui

# Creo un nuevo hyring_dynamics
New-Item -Path "dags/hiring_dynamic_functions.py" -ItemType "File"


# hiring_dynamic_functions.py

#Librerías
from pathlib import Path
import shutil
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score


# Primera definición: create_folders(**context) --> Crea YYYY-MM-DD/(raw, preprocessed, splits, models)

def create_folders(**context):
    ds = context["ds"]                               
    dags_dir = Path(__file__).resolve().parent       
    run_dir  = dags_dir / ds

    for sub in ["raw", "preprocessed", "splits", "models"]:
        (run_dir / sub).mkdir(parents=True, exist_ok=True)

# Segunda definición: load_and_merge(**context) --> Lee raw/data_1.csv (+ data_2.csv si existe), concat y guarda preprocessed/merged.csv

def load_and_merge(**context):
    ds = context["ds"]
    base = Path(__file__).resolve().parent / ds
    raw_dir = base / "raw"
    prep_dir = base / "preprocessed"
    prep_dir.mkdir(parents=True, exist_ok=True)

    p1 = raw_dir / "data_1.csv"
    p2 = raw_dir / "data_2.csv"

    dfs = []
    if p1.exists():
        dfs.append(pd.read_csv(p1))
        print(f"[load_and_merge] Leído: {p1}")
    if p2.exists():
        dfs.append(pd.read_csv(p2))
        print(f"[load_and_merge] Leído: {p2}")

    merged = pd.concat(dfs, ignore_index=True)
    out = prep_dir / "merged.csv"
    merged.to_csv(out, index=False)
    print(f"[load_and_merge] Guardado: {out} (n={len(merged)})")


# Tercera definición: split_data(**context) --> Lee preprocessed/merged.csv y hace hold-out 80/20 (seed=42)

def split_data(**context):
    ds = context["ds"]
    base = Path(__file__).resolve().parent / ds
    merged_path = base / "preprocessed" / "merged.csv"

    df = pd.read_csv(merged_path)

    y = df["HiringDecision"]
    X = df.drop(columns=["HiringDecision"])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.20, random_state=42, stratify=y
    )

    splits_dir = base / "splits"
    splits_dir.mkdir(parents=True, exist_ok=True)
    (X_train.assign(HiringDecision=y_train)).to_csv(splits_dir / "train.csv", index=False)
    (X_test.assign(HiringDecision=y_test)).to_csv(splits_dir / "test.csv", index=False)


# Cuarta definición: train_model(model, model_name=None, **context) --> Entrena pipeline(preproc + modelo) y guarda models/<nombre>.joblib
# Acá también defino el preprocessor de las bases .csv

def _build_preprocessor(X: pd.DataFrame) -> ColumnTransformer:
    """
    Devuelve un ColumnTransformer minimalista (compatible con sklearn 1.3.x).
    """
    num_cols = [
        "Age", "ExperienceYears", "PreviousCompanies", "DistanceFromCompany",
        "InterviewScore", "SkillScore", "PersonalityScore",
    ]
    cat_cols = ["Gender", "EducationLevel", "RecruitmentStrategy"]

    num_cols = [c for c in num_cols if c in X.columns]
    cat_cols = [c for c in cat_cols if c in X.columns]

    numeric = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
    ])

    categorical = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ])

    return ColumnTransformer(
        transformers=[
            ("num", numeric, num_cols),
            ("cat", categorical, cat_cols),
        ],
        remainder="drop",
    )

def train_model(model, model_name: str | None = None, **context):
    """
    Entrena (preprocesamiento + modelo) sobre <ds>/splits/train.csv
    y guarda el pipeline en <ds>/models/<nombre>.joblib
    - model: estimador sklearn ya instanciado (clasificador)
    - model_name: nombre del archivo .joblib (opcional). Si no, usa la clase del modelo.
    """
    ds = context["ds"]
    base = Path(__file__).resolve().parent / ds
    splits_dir = base / "splits"
    models_dir = base / "models"
    models_dir.mkdir(parents=True, exist_ok=True)

    train_path = splits_dir / "train.csv"

    train_df = pd.read_csv(train_path)
    target_col = "HiringDecision"

    y_train = train_df[target_col]
    X_train = train_df.drop(columns=[target_col])

    preprocessor = _build_preprocessor(X_train)

    pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", model),
    ])

    pipe.fit(X_train, y_train)

    base_name = model_name or model.__class__.__name__
    out_path = models_dir / f"{base_name}.joblib"
    joblib.dump(pipe, out_path)
    print(f"[train_model] Modelo entrenado y guardado en: {out_path}")


# Quinta definición: evaluate_models(**context) --> Evalúa todos los .joblib en models/ con accuracy en test y selecciona el mejor y lo guarda como models/best_model.joblib

def evaluate_models(**context):
    ds = context["ds"]
    base = Path(__file__).resolve().parent / ds
    splits_dir = base / "splits"
    models_dir = base / "models"

    test_path = splits_dir / "test.csv"

    test_df = pd.read_csv(test_path)

    y_test = test_df["HiringDecision"]
    X_test = test_df.drop(columns=["HiringDecision"])

    candidates = sorted([p for p in models_dir.glob("*.joblib") if p.name != "best_model.joblib"])

    best_name = None
    best_score = -1.0
    best_pipe = None

    for path in candidates:
        pipe = joblib.load(path)
        y_pred = pipe.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        print(f"[evaluate_models] {path.name}: accuracy={acc:.4f}")
        if acc > best_score:
            best_score = acc
            best_name = path.name
            best_pipe = pipe

    out_best = models_dir / "best_model.joblib"
    joblib.dump(best_pipe, out_best)
    print(f"[evaluate_models] Mejor modelo: {best_name} | accuracy={best_score:.4f}")
    print(f"[evaluate_models] Guardado como: {out_best}")

## **2.2 Componiendo un nuevo DAG** (15 puntos)

<p align="center">
  <img src="https://67.media.tumblr.com/bfa5208006dc3f404ec08e8c3195cf2c/tumblr_obg9tgnLfX1u9e9f2o2_r1_500.gif" width="500">
</p>

Con las nuevas funciones, se debe crear el nuevo nuevo DAG. Para ello, cree un nuevo script en la carpeta `dags`, llamandolo `dag_dynamic.py`. Este script debe contener la siguiente estructura:

1. (1 punto) Inicialice un DAG con fecha de inicio el 1 de octubre de 2024, el cual se debe ejecutar el día 5 de cada mes a las 15:00 UTC. Utilice un `dag_id` interpretable para identificar fácilmente. **Habilite el backfill** para que pueda ejecutar tareas programadas desde fechas pasadas.
2. (1 punto) Comience con un marcador de posición que indique el inicio del pipeline.
3. (2 puntos) Cree una carpeta correspondiente a la ejecución del pipeline y cree las subcarpetas `raw`, `preprocessed`, `splits` y `models` mediante la función `create_folders()`.
4. (2 puntos) Implemente un `Branching`que siga la siguiente lógica:
  - Fechas previas al 1 de noviembre de 2024: Se descarga solo `data_1.csv`
  - Desde el 1 de noviembre del 2024: descarga `data_1.csv` y `data_2.csv`.
  En el siguiente [enlace](https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_2.csv) puede descargar `data_2.csv`.
5. (1 punto) Cree una tarea que concatene los datasets disponibles mediante la función `load_and_merge()`. Configure un `Trigger` para que la tarea se ejecute si encuentra disponible **como mínimo** uno de los archivos.
6. (1 punto) Aplique el hold out al dataset mediante la función `split_data()`, obteniendo un conjunto de entrenamiento y uno de prueba.
7. (2 puntos) Realice 3 entrenamientos en paralelo:
  - Un modelo Random Forest.
  - 2 modelos a elección.
  Asegúrese de guardar sus modelos entrenados con nombres distintivos. Utilice su función `train_model()` para ello.
8. (2 puntos) Mediante la función `evaluate_models()`, evalúe los modelos entrenados, registrando el accuracy de cada modelo en el set de prueba. Luego debe imprimir el mejor modelo seleccionado y su respectiva métrica. Configure un `Trigger` para que la tarea se ejecute solamente si los 3 modelos fueron entrenados y guardados.

`Hint:` Recuerde que puede entregar `kwargs` a sus funciones, como por ejemplo la fecha de ejecución `ds`.

Una vez creado el script, vuelva a construir el contenedor en Docker, acceda a la aplicación web de Airflow, ejecute su pipeline y muestre sus resultados. Adjunte imágenes que ayuden a mostrar el proceso y sus resultados.

Adicionalmente, responda (1 c/u):

- ¿Cual es el accuracy de cada modelo en la ejecución de octubre? ¿Se obtienen los mismos resultados a partir de Noviembre?
- Analice como afectó el añadir datos a sus modelos mediante el desempeño del modelo y en costo computacional.
- Muestre el esquema de su DAG ejecutado en octubre y en noviembre.


`Nota:` Para esta sección no debe implementar la tarea en gradio, solamente se espera determinar el mejor modelo y comparar el desempeño obtenido.

**IMPORTANTE: Para esta sección, debe adjuntar todos los scripts creados junto a su notebook en la entrega, ya que serán ejecutados para validar el funcionamiento. Para justificar sus respuestas, adicionaslmente puede utilizar imágenes de apoyo, como screenshots.**

In [ ]:
#Inserte código aqui

# Creo un nuevo file dag_dynamics
New-Item -Path "dags/dag_dynamic.py" -ItemType "File"

# dag_dynamic.py

# Librerías
from datetime import datetime
import pendulum
from airflow import DAG
from airflow.operators.empty import EmptyOperator
from airflow.operators.bash import BashOperator
from airflow.operators.python import PythonOperator, BranchPythonOperator
from airflow.utils.trigger_rule import TriggerRule
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from hiring_dynamic_functions import (create_folders, load_and_merge, split_data, train_model, evaluate_models)


# DAG --> corre el día 5 de cada mes a las 15:00 UTC: backfill HABILITADO (catchup=True) y start_date = 2024-10-01

with DAG(
    dag_id="hiring_dynamic",               
    start_date=datetime(2024, 10, 1),
    schedule="0 15 5 * *",                   # 5 de cada mes, 15:00 UTC
    catchup=True,                            # habilita backfill
    tags=["lab9", "airflow", "dynamic"],
    is_paused_upon_creation=False,
) as dag:

# Marcador de inicio
    start = EmptyOperator(task_id="start")

# Carpetas YYYY-MM-DD/{raw, preprocessed, splits, models}
    t_create = PythonOperator(
        task_id="create_folders",
        python_callable=create_folders,
        op_kwargs={"ds": "{{ ds }}"},
    )

# Branching por fecha de ejecución, antes del 2024-11-01: solo data_1.csv y desde 2024-11-01 inclusive: data_1.csv y data_2.csv
def branching(**context):
    logical_date = context["logical_date"].in_timezone("UTC")
    cutoff = pendulum.datetime(2024, 11, 1, tz="UTC")
    if logical_date < cutoff:
        return "download_data1"

    return ["download_data1", "download_data2"]

branch = BranchPythonOperator(
    task_id="branch_by_date",
    python_callable=branching,
)

# Descargo de als baases de datos según branching
t_dl1 = BashOperator(
    task_id="download_data1",
    bash_command=(
        "mkdir -p {{ dag.folder }}/{{ ds }}/raw && "
        "curl -sSL -o {{ dag.folder }}/{{ ds }}/raw/data_1.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_1.csv"
    ),
)

t_dl2 = BashOperator(
    task_id="download_data2",
    bash_command=(
        "mkdir -p {{ dag.folder }}/{{ ds }}/raw && "
        "curl -sSL -o {{ dag.folder }}/{{ ds }}/raw/data_2.csv "
        "https://gitlab.com/eduardomoyab/laboratorio-13/-/raw/main/files/data_2.csv"
    ),
)

# Merge de datasets disponibles, debe correr si hay al menos UNO disponible → TriggerRule.ONE_SUCCESS
t_merge = PythonOperator(
        task_id="load_and_merge",
        python_callable=load_and_merge,
        op_kwargs={"ds": "{{ ds }}"},
        trigger_rule=TriggerRule.ONE_SUCCESS,
    )

# split hold-out (80/20 con semilla) → splits/train.csv y splits/test.csv
t_split = PythonOperator(
        task_id="split_data",
        python_callable=split_data,
        op_kwargs={"ds": "{{ ds }}"},
    )

# Tres entrenamientos en paralelo
t_train_rf = PythonOperator(
        task_id="train_rf",
        python_callable=train_model,
        op_kwargs={
            "ds": "{{ ds }}",
            "model": RandomForestClassifier(n_estimators=300, random_state=42),
            "model_name": "RandomForest",
        },
    )

t_train_lr = PythonOperator(
        task_id="train_lr",
        python_callable=train_model,
        op_kwargs={
            "ds": "{{ ds }}",
            "model": LogisticRegression(max_iter=1000),
            "model_name": "LogReg",
        },
    )

t_train_dt = PythonOperator(
        task_id="train_dt",
        python_callable=train_model,
        op_kwargs={
            "ds": "{{ ds }}",
            "model": DecisionTreeClassifier(random_state=42),
            "model_name": "DecisionTree",
        },
    )

# Evaluación del mejor modelo, aca se va a correr solo si los 3 entrenamientos terminaron OK → ALL_SUCCESS si no, no porque falla
t_eval = PythonOperator(
        task_id="evaluate_models",
        python_callable=evaluate_models,
        op_kwargs={"ds": "{{ ds }}"},
        trigger_rule=TriggerRule.ALL_SUCCESS,
    )

end = EmptyOperator(task_id="end")

# Estructura del DAG
start >> t_create >> branch
branch >> t_dl1
branch >> t_dl2
[t_dl1, t_dl2] >> t_merge >> t_split >> [t_train_rf, t_train_lr, t_train_dt] >> t_eval >> end

In [ ]:
## Vuelvo a crear un nuevo dockerfile.dockerfile
# Eset es mi código del archivo Dockerfile_dynamics.dockerfile (no va a correr aca obviamente al no ser .py)

# Me redirecciono a mi carpte
cd Laboratorios\Laboratorio_9

# Creación del dockerfile
New-Item -Path . -Name "Dockerfile_dynamics" -ItemType "File"


# Dockerfile_dynamics
FROM python:3.10-slim

ENV DEBIAN_FRONTEND=noninteractive \
    PYTHONUNBUFFERED=1 \
    AIRFLOW_HOME=/opt/airflow

# Requisitos de SO (incluye curl como pide el enunciado)
RUN apt-get update && apt-get install -y --no-install-recommends \
    curl build-essential git && \
    rm -rf /var/lib/apt/lists/*

# Airflow con constraints para Py3.10
RUN pip install --no-cache-dir "apache-airflow==2.10.2" \
  --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.10.2/constraints-3.10.txt"

# Paquetes de tu pipeline (pin de sklearn para compat con sparse_output=False)
RUN pip install --no-cache-dir \
    pandas==2.2.2 scikit-learn==1.3.2 joblib numpy \
    gradio

# Estructura estándar de Airflow
RUN mkdir -p ${AIRFLOW_HOME}/dags ${AIRFLOW_HOME}/logs ${AIRFLOW_HOME}/plugins
WORKDIR ${AIRFLOW_HOME}

# (Opcional) Copiar dags en build; normalmente los montaremos por volumen al correr
# COPY dags/ ./dags/

EXPOSE 8080 7860
CMD ["bash", "-lc", "airflow standalone"]


##-------------------------------------------------------------------------------------------------------------------------------------------------------------
## CLAVE ABRIR/ENCENDER DOCKER DESKTOP O SIMILAR

# Primero hay que estar parado en la carpeta, si no se esta ahí use este comentario sin el #
# cd Laboratorios\Laboratorio_9

# Tuve que especificar el nmbre de mi archivo pq por defecto los guarda con extension y docker no reconcoa eso me daba error

# Build imagen dinámica
docker build -f Dockerfile_dynamics -t airflow-hiring:latest .

# Elimina contenedor previo
docker rm -f hiring-airflow 2>$null

# Crea y levanta Airflow
docker run --name hiring-airflow `
  -p 8080:8080 `
  -e AIRFLOW__CORE__LOAD_EXAMPLES=False `
  -e AIRFLOW__CORE__EXECUTOR=SequentialExecutor `
  -e AIRFLOW__WEBSERVER__BASE_URL=http://localhost:8080 `
  -v "$(pwd)/dags:/opt/airflow/dags" `
  -v "$(pwd)/logs:/opt/airflow/logs" `
  -v "$(pwd)/plugins:/opt/airflow/plugins" `
  airflow-hiring:latest

# Web para revisar
http://localhost:8080


# Preguntas adicionales

#### 1. Muestre el esquema de su DAG ejecutado en octubre y en noviembre

**Flujo del DAG del hiring_dynamic.py --> octubre y noviemrbe**

**OCTUBRE**

![Flujo DAG octubre](DAGoct.png)
![Diagrama octubre](diag_oct_2024.png)

**NOVIEMBRE**

![Flujo DAG noviembre](DAGnov.png)
![Diagrama noviembre](diag_nov_2024.png)

**Comprobación del flujo global**
![Flujo global correcto](comprobacion_global_que_funciona.png)

#### 2. ¿Cual es el accuracy de cada modelo en la ejecución de octubre? ¿Se obtienen los mismos resultados a partir de Noviembre?

**OCTUBRE**

![Métricas mejor modelo octubre](metricas_mejor_modelo_oct.png)

**NOVIEMBRE**

![Métricas mejor modelo noviembre](metricas_mejor_modelo_nov.png)

**Respuesta:** En la ejecución de octubre (solo `data_1.csv`), los accuracy fueron:

* DecisionTree: **0.8667**
* LogReg: **0.9133**
* RandomForest: **0.9267** → mejor modelo.

En noviembre (con `data_1.csv` + `data_2.csv`), los resultados fueron:

* DecisionTree: **0.8800**
* LogReg: **0.9100**
* RandomForest: **0.9267** → nuevamente el mejor.

Los desempeños se mantuvieron prácticamente iguales, con una leve mejora en los modelos más simples y sin variación en el Random Forest, que siguió siendo el más preciso.

Además, aunque no se muestran los logs de `evaluate_models` para los demás meses, las ejecuciones posteriores confirman que a partir de noviembre los valores se estabilizan, ya que el dataset no cambia.  
Si en el futuro se incorporaran nuevos datos, podrían observarse variaciones menores, pero con la evidencia actual el desempeño de los modelos se mantiene constante.

### 3. Analice cómo afectó el añadir datos a sus modelos mediante el desempeño del modelo y en costo computacional

**Respuesta:**

Al incorporar el archivo `data_2.csv` a partir de noviembre, el pipeline `hiring_dynamic.py` procesó un mayor volumen de datos sin afectar el rendimiento general del modelo.  
En ambas ejecuciones (octubre y noviembre), el **Random Forest** se mantuvo como el mejor clasificador con un *accuracy* estable de **0.9267**, seguido por Logistic Regression (~0.91) y Decision Tree (~0.88).  
Esto demuestra que el modelo es **robusto y consistente frente al aumento de datos**, sin pérdida de precisión ni sobreajuste.

En términos de **costo computacional**, el impacto fue mínimo.  
El tiempo total del DAG aumentó levemente —de **00:04:15** en octubre a **00:05:13** en noviembre—, lo cual era esperable ya que en noviembre se ejecuta también la descarga de `data_2.csv` y su fusión en la etapa `load_and_merge`.  
El resto de las tareas (entrenamiento, evaluación y splits) se mantuvo prácticamente igual, confirmando que el pipeline escala bien y mantiene buena eficiencia incluso con más registros.

Además, al habilitar el **backfill (`catchup=True`)** en el DAG dinámico, Airflow ejecutó automáticamente **todas las fechas pasadas programadas** desde el `start_date` definido (1 de octubre de 2024) hasta la fecha actual, corriendo las queries históricas una por una.  
Esto permitió observar cómo el DAG se comporta mes a mes y cómo gestiona el histórico completo, lo que es clave para flujos que se mantienen activos en producción y procesan datos acumulativos.  
El backfill no solo valida la consistencia del pipeline en fechas pasadas, sino que también asegura que **todas las corridas pendientes se ejecuten sin intervención manual**, lo cual es exactamente el propósito de un flujo dinámico.

Las siguientes figuras muestran los tiempos promedio por tarea en las ejecuciones de octubre y noviembre:

#### Creación y branching inicial
![Duración create_folders](costo_computacional_create_folders.png)
![Duración branch_by_date](costo_computacional_branch.png)

#### Descarga y preparación de datos
![Duración download_data1](costo_computacional_data1.png)
![Duración download_data2](costo_computacional_data2.png)
![Duración load_and_merge](costo_computacional_loadmerge.png)
![Duración split_data](costo_computacional_splitdata.png)

#### Entrenamiento de modelos
![Duración train_rf](costo_computacional_rf.png)
![Duración train_lr](costo_computacional_lr.png)
![Duración train_dt](costo_computacional_dt.png)

#### Evaluación final
![Duración evaluate_models](costo_computacional_eval_modelos.png)

En conjunto, los resultados globales de ejecución fueron los siguientes:

| Fecha de ejecución | Duración total del DAG | Estado final |
|--------------------|------------------------|---------------|
| 2024-10-05 | **00:04:15** | `success` |
| 2024-11-05 | **00:05:13** | `success` |

El aumento de datos generó un incremento de alrededor de **1 minuto** en la duración total del pipeline, atribuible casi por completo a la etapa de descarga y concatenación de archivos, mientras que los tiempos de entrenamiento (entre **30 y 55 s**) se mantuvieron estables entre corridas.  

Esto confirma que el pipeline **es eficiente, reproducible y escalable**, manteniendo el desempeño del modelo y el comportamiento temporal controlado aun cuando crece el volumen de datos.

**OCTUBRE**

![Duración total del DAG – octubre](diag_oct_2024.png)

**NOVIEMBRE**

![Duración total del DAG – noviembre](diag_nov_2024.png)


# Conclusión

Éxito!
<div align="center">
  <img src="https://miro.medium.com/v2/resize:fit:1000/1*PX8WVijZapo7EDrvGv9Inw.gif" width="500">
</div>
